# Function Registry

_Register functions to enable chat models to call them._

The hidden power behind `chat.register(ƒ)` is the Function Registry. It lets you register _typed_ functions quickly and easily so they can be called on behalf of chat models.


In [1]:
from chatlab import FunctionRegistry, Chat, expose_exception_to_llm
from typing import Union, List
from IPython.display import JSON


def f(x: float):
    """Multiply x by 2."""
    return x * 2


registry = FunctionRegistry()
registry.register_function(f)

await registry.call("f", '{"x": 4}')

8

## Registering via Decorators

`register` is also a decorator, allowing you to register a function while declaring it.


In [2]:
from typing import Tuple, Optional
import math


@registry.register
def quadratic_formula(a: float, b: float, c: float) -> Optional[Tuple[float, float]]:
    """
    Solves the quadratic equation ax^2 + bx + c = 0.
    :param a: Coefficient of x^2
    :param b: Coefficient of x
    :param c: Constant term
    :return: Tuple containing two solutions or None if no real solutions exist
    """
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        return None  # No real solutions
    x1 = (-b + math.sqrt(discriminant)) / (2 * a)
    x2 = (-b - math.sqrt(discriminant)) / (2 * a)
    return x1, x2


await registry.call("quadratic_formula", '{"a": 1, "b": 0, "c": -1}')

(1.0, -1.0)

Functions can be registered by:

- Passing a typed function in directly
- Passing the function and the Pydantic model that describes its parameters
- Passing the function and a JSON schema that describes its parameters

## Registering Typed Functions

> [!NOTE]
> Not all function parameters can be accurately determined. For those cases it's best to use a Pydantic model or JSON schema.
>
> If you're adventurous, make a [PR to chatlab](https://github.com/rgbkrk/chatlab) to improve the type inference!

### No Parameters


In [3]:
import random


def flip_a_coin():
    """Flip a coin."""
    return random.choice(["heads", "tails"])


registry.register(flip_a_coin)

FunctionDefinition(name='flip_a_coin', parameters={'properties': {}, 'type': 'object', 'required': []}, description='Flip a coin.')

### Multiple Parameters


In [4]:
def add(x: float, y: float):
    """Add x and y."""
    return x + y


registry.register(add)

FunctionDefinition(name='add', parameters={'properties': {'x': {'type': 'number'}, 'y': {'type': 'number'}}, 'required': ['x', 'y'], 'type': 'object'}, description='Add x and y.')

### Optional Parameters


In [5]:
def add(x: float, y: float = 1):
    """Add x and y."""
    return x + y


registry.register(add)

FunctionDefinition(name='add', parameters={'properties': {'x': {'type': 'number'}, 'y': {'default': 1, 'type': 'number'}}, 'required': ['x'], 'type': 'object'}, description='Add x and y.')

## Registering Functions with Pydantic Models

Pydantic Models give you all the benefits above _and_ much more:

- Default values
- Descriptions on parameters
- Deeper type validation


In [6]:
from chatlab import FunctionRegistry
from pydantic import BaseModel

registry = FunctionRegistry()


class CompoundInterestModel(BaseModel):
    principal: float
    rate: float
    times_compounded: int
    years: float


@registry.register(parameter_schema=CompoundInterestModel)
def compound_interest(principal: float, rate: float, times_compounded: int, years: float) -> float:
    """
    Calculates the future value of an investment using compound interest.
    :param principal: Initial investment amount
    :param rate: Annual interest rate (as a decimal)
    :param times_compounded: Number of times interest is compounded per year
    :param years: Number of years the money is invested
    :return: Future value of the investment
    """
    return principal * (1 + rate / times_compounded) ** (times_compounded * years)


await registry.call(
    "compound_interest",
    CompoundInterestModel(principal=1000, rate=0.05, times_compounded=12, years=5).model_dump_json(),
)

/var/folders/r4/lz3zpzb528n55t008k22p4dh0000gn/T/ipykernel_79757/187773287.py:28: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  "compound_interest", CompoundInterestModel(principal=1000, rate=0.05, times_compounded=12, years=5).json()


1283.3586785035118

### Scratch Space

If you made it this far, **you're probably ready to start writing your own functions**. This notebook is a scratch space for you to try out your functions and make sure they work as expected. You can also use this space to test out the functions that are already in the registry. That's what I'm going to do in the following cells.


In [7]:
registry = FunctionRegistry()

@registry.register
@expose_exception_to_llm
def plussle(x: Union[int, float]):
    """Run this and you might find the secret constant added to x."""
    return x + 42


@registry.register
@expose_exception_to_llm
def check_secret_number(x: Union[int, float]):
    """Check if the secret number is correct."""
    return x == 42


print(registry.api_manifest())


chat = Chat(
    function_registry=registry,
)
await chat("Run plussle with a number to determine the secret constant.")
await chat("Verify the secret you determined.")
await chat()
await chat("Critique your performance.")
await chat()

{'functions': [{'name': 'plussle', 'parameters': {'properties': {'x': {'anyOf': [{'type': 'integer'}, {'type': 'number'}]}}, 'required': ['x'], 'type': 'object'}, 'description': 'Run this and you might find the secret constant added to x.'}, {'name': 'check_secret_number', 'parameters': {'properties': {'x': {'anyOf': [{'type': 'integer'}, {'type': 'number'}]}}, 'required': ['x'], 'type': 'object'}, 'description': 'Check if the secret number is correct.'}], 'function_call': 'auto'}


The secret constant added to the number 5 is 47.

The secret constant I determined is incorrect.

I apologize for the incorrect secret constant. It seems that the secret number is not 47.

I apologize for the incorrect determination of the secret constant. It seems that my initial guess was incorrect. I will need to reassess and try again to determine the correct secret constant. Thank you for bringing this to my attention.

I apologize for the incorrect determination of the secret constant. It seems that my initial guess was incorrect. I will need to reassess and try again to determine the correct secret constant. Thank you for bringing this to my attention.

In [8]:
registry = FunctionRegistry()

@registry.register
@expose_exception_to_llm
def plussle2(x: Optional[Union[int, float]] = None):
    """Run this and you might find the secret constant added to x."""
    if x is None:
        return "You need to pass a number to plussle2."

    return x + 42


@registry.register
@expose_exception_to_llm
def check_secret_number2(x: Union[int, float]):
    """Check if the secret number is correct."""
    return x == 42


print(registry.api_manifest())


chat = Chat(
    function_registry=registry,
)
await chat("Run plussle with a number to determine the secret constant.")
await chat("Verify the secret you determined.")
await chat()
await chat("Critique your performance.")
await chat()

{'functions': [{'name': 'plussle', 'parameters': {'properties': {'x': {'anyOf': [{'type': 'integer'}, {'type': 'number'}, {'type': 'null'}], 'default': None}}, 'type': 'object', 'required': []}, 'description': 'Run this and you might find the secret constant added to x.'}, {'name': 'check_secret_number', 'parameters': {'properties': {'x': {'anyOf': [{'type': 'integer'}, {'type': 'number'}]}}, 'required': ['x'], 'type': 'object'}, 'description': 'Check if the secret number is correct.'}], 'function_call': 'auto'}


The secret constant added to the number 5 is 47.

I apologize, but it seems that the secret constant I determined is incorrect.

The secret constant added to the number 10 is 52.

I apologize for the incorrect determination of the secret constant in my previous response. It seems that my initial guess was incorrect. I will try to improve my performance and provide more accurate results in the future. Thank you for bringing this to my attention.

I apologize for the incorrect determination of the secret constant. It seems that my initial guess was incorrect. I understand that accuracy is crucial in providing reliable information, and I apologize for any confusion or inconvenience caused. I will make sure to improve my performance and provide more accurate results in the future. Thank you for bringing this to my attention, and I appreciate your understanding.

In [9]:
registry = FunctionRegistry()


@registry.register
def poppy(x: list):
    """pop off"""
    return x[0]


await registry.call("poppy", '{"x": [1, 2, 3]}')

1

In [10]:
registry = FunctionRegistry()


@registry.register
def pop_off(x: List[Union[float, int]]):
    """another"""
    return x[0]


print(registry.get_schema("pop_off"))

await registry.call("pop_off", '{"x": [1, 2, 3]}')

FunctionDefinition(name='poppy', parameters={'properties': {'x': {'items': {'anyOf': [{'type': 'number'}, {'type': 'integer'}]}, 'type': 'array'}}, 'required': ['x'], 'type': 'object'}, description='another')


1

In [11]:
registry = FunctionRegistry()

@registry.register
@expose_exception_to_llm
def example(data: dict):
    """testing if the llm can figure out the parameters"""
    return data.keys()


print(registry.get_schema("example"))

print(registry.api_manifest())

chat = Chat(function_registry=registry)

await chat("can you call `example` with something valid? Please figure out what datatype works")

FunctionDefinition(name='example', parameters={'properties': {'data': {'type': 'object'}}, 'required': ['data'], 'type': 'object'}, description='testing if the llm can figure out the parameters')
{'functions': [{'name': 'example', 'parameters': {'properties': {'data': {'type': 'object'}}, 'required': ['data'], 'type': 'object'}, 'description': 'testing if the llm can figure out the parameters'}], 'function_call': 'auto'}


It seems that the `example` function requires a parameter called `data`. Let me try calling it with a valid value.

Apologies for the confusion. It seems that the `example` function expects the `data` parameter to be an object with keys. Let me try calling it with a valid object.

The `example` function was successfully called with a valid object. It returned the keys of the object, which are `name` and `age`.

In [13]:
from enum import Enum, auto


class CellType(Enum):
    markdown = auto()
    code = auto()

registry = FunctionRegistry()


@registry.register
@expose_exception_to_llm
def create_cell(cell_type: CellType, source: str, metadata: Optional[dict] = None) -> dict:
    """Create a cell"""
    return {"cell_type": cell_type, "source": source, "metadata": metadata}


display(JSON(registry.api_manifest()))

chat = Chat(function_registry=registry)

await chat("Create a markdown cell describing this chat and then a code cell that does a simple print statement")

<IPython.core.display.JSON object>

I have created a markdown cell that describes this chat and a code cell that contains a simple print statement.